In [1]:
import torch
from torchvision.models import vgg19
from torchvision import transforms
import torch
from torch.utils.data import Dataset, DataLoader

from PIL import Image

import numpy as np
import torch
import os
from PIL import Image, ImageOps, ImageFilter
import random
import time
import pandas as pd


In [2]:
os.listdir("./Artifact")

['afhq',
 'big_gan',
 'celebahq',
 'cips',
 'coco',
 'cycle_gan',
 'ddpm',
 'denoising_diffusion_gan',
 'diffusion_gan',
 'face_synthetics',
 'ffhq',
 'gansformer',
 'gau_gan',
 'generative_inpainting',
 'glide',
 'imagenet',
 'lama',
 'landscape',
 'latent_diffusion',
 'lsun',
 'mat',
 'metfaces',
 'palette',
 'projected_gan',
 'pro_gan',
 'sfhq',
 'stable_diffusion',
 'star_gan',
 'stylegan1',
 'stylegan2',
 'stylegan3',
 'taming_transformer',
 'vq_diffusion']

In [9]:
dataDir = "./Artifact/"
tsfms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor()
])

In [3]:
trainDataLoader, validDataLoader, testDataLoader = ImageData.getImagesDataLoader(dataDir, batchSize=32, transforms = tsfms)

In [17]:
class ImageData(Dataset):
    test_generators = ['ddpm', 'gau_gan', 'pallet', 'cips']
    real_data = ['afhq', 'celebahq', 'coco', 'imagenet', 'landscape', 'lsun', 'metfaces', '']

    def __init__(self, dataParentFolder: str, dataIdxs: list = None, transform = None, batch_size=128):
        self.dataParentFolder = dataParentFolder
        self.transform = transform

        imagePaths = []
        imageClass = []
        for dataset in os.listdir(dataParentFolder):
            path = os.path.join(dataParentFolder, dataset)
            df = pd.read_csv(os.path.join(path, 'metadata.csv'))
            imageFiles = df['image_path'].tolist()
            imagePaths.extend([os.path.join(path, imageFile).replace("\\", "/") for imageFile in imageFiles])
            imageClass.append(dataset)

        self.imagePaths = np.array(imagePaths)
        self.imageClass = np.array(imageClass)

        if dataIdxs:
            self.imagePaths = self.imagePaths[dataIdxs]
            self.imageClass = self.imageClass[dataIdxs]

    def __len__(self):
        return len(self.imagePaths)

    def __getitem__(self, idx: int):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        imagePath = self.imagePaths[idx]
        pathSplits = imagePath.split(self.dataParentFolder)[1].split("/")
        dataset = pathSplits[0]
        dataset =self.image_to_vector(dataset)
        image = Image.open(imagePath)
        if self.transform:
            image = self.transform(image)
    
        return image, dataset
    
    # TODO: 
    def image_to_vector(self, dataset):
        labels = {
            "afhq":0,
            "big_gan":1,
            "celebahq":2,
            "cips":3,
            "coco":4,
            "cycle_gan":5,
            "ddpm":6,
            "denoising_diffusion_gan":7,
            "diffusion_gan":8,
            "face_synthetics":9,
            "ffhq":10,
            "gansformer":11,
            "gau_gan":12,
            "generative_inpainting":13,
            "glide":14,
            "imagenet":15,
            "lama":16,
            "landscape":17,
            "latent_diffusion":18,
            "lsun":19,
            "mat":20,
            "metfaces":21,
            "palette":22,
            "projected_gan":23,
            "pro_gan":24,
            "sfhq":25,
            "stable_diffusion":26,
            "star_gan":27,
            "stylegan1":28,
            "stylegan2":29,
            "stylegan3":30,
            "taming_transformer":31,
            "vq_diffusion":32
        }

        return torch.tensor(labels.get(dataset))


In [6]:
# gens = [
            'big_gan',
            'cips',
            'cycle_gan',
            'ddpm',
            'denoising_diffusion_gan',
            'diffusion_gan',
            'face_synthetics',
            'gansformer',
            'gau_gan',
            'generative_inpainting',
            'glide',
            'lama',
            'latent_diffusion',
            'mat',
            'palette',
            'projected_gan',
            'pro_gan',
            'sfhq',
            'stable_diffusion',
            'star_gan',
            'stylegan1',
            'stylegan2',
            'stylegan3',
            'taming_transformer',
            'vq_diffusion'
# ]
# count = 0
# for thing in gens:
#     print(f"\"{thing}\":{count},")
#     count+=1

In [18]:
trainDatset = ImageData(dataDir, transform=tsfms)

In [20]:
data = trainDatset.__getitem__(0)

In [28]:
trainDataLoader = DataLoader(trainDatset, batch_size=8)

In [29]:
for thing in trainDataLoader:
    print(thing)
    break

[tensor([[[[0.9059, 0.9020, 0.8902,  ..., 0.5333, 0.5608, 0.5922],
          [0.9020, 0.9020, 0.8941,  ..., 0.5647, 0.5725, 0.5843],
          [0.8941, 0.8980, 0.8941,  ..., 0.5882, 0.5843, 0.5804],
          ...,
          [0.6706, 0.6588, 0.6471,  ..., 0.1490, 0.1647, 0.1725],
          [0.6784, 0.6627, 0.6471,  ..., 0.1490, 0.1647, 0.1765],
          [0.6745, 0.6588, 0.6392,  ..., 0.1529, 0.1686, 0.1804]],

         [[0.9255, 0.9216, 0.9176,  ..., 0.5647, 0.5882, 0.6196],
          [0.9216, 0.9216, 0.9216,  ..., 0.6000, 0.6000, 0.6118],
          [0.9216, 0.9255, 0.9255,  ..., 0.6235, 0.6118, 0.6078],
          ...,
          [0.7333, 0.7176, 0.6980,  ..., 0.1020, 0.1059, 0.1137],
          [0.7373, 0.7216, 0.7020,  ..., 0.1020, 0.1059, 0.1137],
          [0.7333, 0.7176, 0.6941,  ..., 0.1020, 0.1059, 0.1176]],

         [[0.9373, 0.9333, 0.9294,  ..., 0.5922, 0.6118, 0.6431],
          [0.9333, 0.9333, 0.9294,  ..., 0.6275, 0.6314, 0.6353],
          [0.9294, 0.9333, 0.9333,  ..., 

In [37]:
trainDatset.__getitem__(2000000)[1]

tensor(29)

In [4]:
for thing in trainDataLoader:
    print(thing)
    break

[tensor([[[[0.0431, 0.0510, 0.0549,  ..., 0.0824, 0.0824, 0.0784],
          [0.0431, 0.0510, 0.0549,  ..., 0.0902, 0.0902, 0.0902],
          [0.0431, 0.0510, 0.0510,  ..., 0.0941, 0.0902, 0.0863],
          ...,
          [0.9882, 0.9882, 0.9882,  ..., 0.9961, 0.9961, 0.9961],
          [0.9882, 0.9882, 0.9882,  ..., 0.9961, 0.9961, 0.9961],
          [0.9882, 0.9882, 0.9882,  ..., 0.9961, 0.9961, 0.9961]],

         [[0.0392, 0.0471, 0.0510,  ..., 0.0941, 0.0902, 0.0902],
          [0.0392, 0.0471, 0.0510,  ..., 0.1020, 0.0980, 0.1020],
          [0.0392, 0.0471, 0.0471,  ..., 0.1059, 0.1020, 0.1020],
          ...,
          [0.9725, 0.9725, 0.9725,  ..., 0.9804, 0.9804, 0.9804],
          [0.9725, 0.9725, 0.9725,  ..., 0.9804, 0.9804, 0.9804],
          [0.9725, 0.9725, 0.9725,  ..., 0.9804, 0.9804, 0.9804]],

         [[0.0706, 0.0784, 0.0824,  ..., 0.1412, 0.1373, 0.1255],
          [0.0706, 0.0784, 0.0824,  ..., 0.1490, 0.1451, 0.1373],
          [0.0706, 0.0784, 0.0784,  ..., 